# Ayala Morales Mauricio
### No. de cuenta: 315332122

---

# 4. Tokenization

## Práctica 4: Subword tokenization
**Fecha de entrega: 19 de Octubre 11:59pm**

### Instalación de dependencias

In [2]:
%%python -m spacy download en_core_web_sm
%pip install sentencepiece
%pip install transformers
%pip install subword-nmt
%pip install elotl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Importación de módulos y bibliotecas

In [3]:
import nltk
from nltk.corpus import brown
nltk.download('brown')
from collections import Counter
import re
import elotl.corpus
import elotl.nahuatl.orthography
import math

[nltk_data] Downloading package brown to /home/mauricio/nltk_data...
[nltk_data]   Package brown is already up-to-date!


- Calcular la entropía de dos textos: brown y axolotl
    - Calcular para los textos tokenizados word-level

Implementando la función para calcular la entropía de un texto. La entropía se define como:

$$H(X) = - \sum_i p(x_i) \cdot \log_2(p(x_i))$$

In [4]:
def calculate_entropy(corpus: list[str]) -> float:
    """
    Calculates the entropy of a given text as a list of words.

    :param corpus: list of strings representing the text.
    :return entropy: Entropy value.
    """
    words_counts = Counter(corpus)
    total_words = len(corpus)
    probabilities = {word: count / total_words for word, count in words_counts.items()}
    entropy = -sum(p * math.log2(p) for p in probabilities.values())
    return entropy

In [5]:
## Dividing each corpus by words
brown_corpus = [word for word in brown.words() if re.match("\w", word)]
brown_entropy_wl = calculate_entropy(brown_corpus)

axolotl = elotl.corpus.load("axolotl")
axolotl_corpus = [word for row in axolotl for word in row[1].lower().split() if re.match("\w", word)]
axolotl_entropy_wl = calculate_entropy(axolotl_corpus)

print("Brown corpus length: ", len(brown_corpus))
print("Axolotl corpus length: ", len(axolotl_corpus))

Brown corpus length:  1012528
Axolotl corpus length:  284046


- Calcular para los textos tokenizados con BPE
        - Tokenizar con la biblioteca `subword-nmt`

In [6]:
CORPORA_PATH = ""

def train_test_split(corpus: list[str], test_size: float) -> tuple[list[str], list[str]]:
    """
    Splits the raw text into train and test sets.

    :param corpus: The raw text to be split.
    :param test_size: The percentage of the text to be used for testing.
    :return: A tuple of two lists of strings, the first one is the train set and the second one is the test set.
    """
    train_text = corpus[:int(len(corpus) * test_size)]
    test_text = corpus[int(len(corpus) * test_size):]
    return train_text, test_text

def write_plain_text_corpus(corpus: str, file_name: str) -> None:
    """
    Writes the corpus to a plain text file.

    :param corpus: The corpus to be written.
    :param file_name: The name of the file.
    :return: None.
    """
    with open(f"{file_name}.txt", "w") as f:
        f.write(corpus)

In [7]:
## Dividing each corpus into training and testing sets
brown_train, brown_test = train_test_split(brown_corpus, test_size=0.4)
axolotl_train, axolotl_test = train_test_split(axolotl_corpus, test_size=0.4)

write_plain_text_corpus(" ".join(brown_train), CORPORA_PATH + "brown_plain")
write_plain_text_corpus(" ".join(axolotl_train), CORPORA_PATH + "axolotl_plain")

In [14]:
# Training the models
!subword-nmt learn-bpe -s 100 < brown_plain.txt > brown.model
!subword-nmt learn-bpe -s 100 < axolotl_plain.txt > axolotl.model

100%|########################################| 100/100 [00:00<00:00, 133.55it/s]


In [15]:
write_plain_text_corpus(" ".join(brown_test), CORPORA_PATH + "brown_plain_test")
write_plain_text_corpus(" ".join(axolotl_test), CORPORA_PATH + "axolotl_plain_test")

In [16]:
# Applying BPE to the test data
!subword-nmt apply-bpe -c brown.model < brown_plain_test.txt > brown_tokenized.txt
!subword-nmt apply-bpe -c axolotl.model < axolotl_plain_test.txt > axolotl_tokenized.txt

In [17]:
with open(CORPORA_PATH + "brown_tokenized.txt") as f:
    brown_test_tokenized = f.read().split()

with open(CORPORA_PATH + "axolotl_tokenized.txt") as f:
    axolotl_test_tokenized = f.read().split()

brown_entropy_bpe = calculate_entropy(brown_test_tokenized)
axolotl_entropy_bpe = calculate_entropy(axolotl_test_tokenized)

print("Brown: ", Counter(brown_test_tokenized).most_common(10))
print("Axolotl: ", Counter(axolotl_test_tokenized).most_common(10))

Brown:  [('t@@', 46827), ('e@@', 44243), ('s@@', 44174), ('e', 43313), ('o@@', 42825), ('m@@', 39443), ('i@@', 39216), ('d@@', 37665), ('p@@', 37463), ('the', 37135)]
Axolotl:  [('i@@', 19526), ('a@@', 18876), ('tla@@', 14929), ('o@@', 14635), ('ti@@', 13784), ('ca@@', 12896), ('te@@', 12555), ('n@@', 11826), ('qui@@', 11743), ('l@@', 11742)]


- Imprimir en pantalla:
    - Entropía de axolotl word-base y bpe
    - Entropía del brown word-base y bpe

In [12]:
print("Brown word-level tokenization entropy:", brown_entropy_wl)
print("Brown BPE tokenization entropy:", brown_entropy_bpe)

print("Axolotl word-level tokenization entropy:", axolotl_entropy_wl)
print("Axolotl BPE tokenization entropy:", axolotl_entropy_bpe)

Brown word-level tokenization entropy: 10.859371415775938
Brown BPE tokenization entropy: 6.704903821809703
Axolotl word-level tokenization entropy: 11.801548813770816
Axolotl BPE tokenization entropy: 6.63685439110489


- Responder las preguntas:
    - ¿Aumento o disminuyó la entropía para los corpus?
        - axolotl: Disminuyó.
        - brown: Disminuyó.
    - ¿Qué significa que la entropía aumente o disminuya en un texto?

        Una mayor entropía significa más aleatoriedad debido a una mayor variedad de símbolos (en este caso tokens). Por lo contrario, una menor entropía significa más predictibilidad debido a que hay una menor variedad de símbolos, por lo tanto mayor reconocimiento de patrones.

    - ¿Como influye la tokenización en la entropía de un texto?

        Como se puede observar en los resultados, el método de tokenización utilizado puede aumentar o disminuir la entropía de un texto. En el caso de la tokenización por palabras (el más simple) cada palabra resulta en un token diferente, lo que eleva el número total de tipos, así como la entropía. En el caso de BPE, se pueden encontrar patrones dentro de palabras, por ejemplo prefijos y sufijos, los cuales son más repetitivos y por lo tanto habrá menor variedad de tipos, así como una menor entropía.

### Referencias:

- [Corpora de la biblia en varios idiomas](https://github.com/ximenina/theturningpoint/tree/main/Detailed/corpora/corpusPBC)
- [Biblioteca nativa para BPE](https://github.com/rsennrich/subword-nmt)
- [Tokenizers Hugging face](https://huggingface.co/docs/transformers/tokenizer_summary)